A notebook to to take the pdf source document and produce a txt version. 
NOTE: The txt version needs manual intervention to correct for formatting issues.
Post the manual preperation of the txt file it is loaded into the data structures that will be used for processing

In [2]:
import re
import fitz
import pandas as pd

In [3]:
import importlib
import src.valid_index
importlib.reload(src.valid_index)
from src.valid_index import get_excon_manual_index

adla_index = get_excon_manual_index()

In [2]:
#pdf_file = "./pdf/Currency and Exchanges Manual for Authorised Dealers_20230523.pdf"
pdf_file = "./pdf/Currency and Exchanges Manual for ADLAs_20231013.pdf"
doc = fitz.open(pdf_file)


In [71]:
def clean_text(text):
    lines = text.split("\n")
    cleaned_lines = []
    for line in lines:
        if line.strip():  # If the line contains text other than whitespace
            line_text = line.strip()
            line_text = line_text.replace("---oOo---", "")
            cleaned_lines.append(line_text)
        else:  # If the line is blank
            if cleaned_lines and cleaned_lines[-1] != "\n":  
                cleaned_lines.append("\n")
    return " ".join(cleaned_lines).replace(" \n ", "\n")
    
def output_doc_as_text(pdf_doc, start_page = 0, end_page = 0, header_size=80, footer_size=70):
    if end_page == 0 or end_page > len(pdf_doc):
        end_page = len(pdf_doc)
    if end_page < start_page:
        print("End page before start page! Doing nothing")
    combined_text = ''
    for page_number in range(start_page, end_page):
        page = pdf_doc[page_number]
        tl = page.rect[0], page.rect[1]  # lower-left coordinates
        br = page.rect[2], page.rect[3]  # upper-right
        rect = fitz.Rect(tl[0], tl[1]+header_size, br[0], br[1]-footer_size)
        raw_text = page.get_text('text', clip=rect)

        cleaned_text = clean_text(raw_text)
        combined_text += cleaned_text
    return combined_text


def get_indent(line, valid_index):
    indent = None
    # Check if line starts with any string from exclusion_list
    if any(line.startswith(exclusion) for exclusion in valid_index.exclusion_list):
        indent = 0
    
    # Check if line starts with a string matching any regex pattern from excon_index_patterns
    if indent is None:
        for idx, pattern in enumerate(valid_index.index_patterns):
            if re.match(pattern, line):
                indent = idx * 4
                break
    return indent

def add_indents(text, valid_index):
    previous_indent = 0
    lines = text.split("\n")
    modified_lines = []
    for line in lines:
        indent = get_indent(line, valid_index)
        if indent is None:
            indent = previous_indent
        modified_line = " " * indent + line
        modified_lines.append(modified_line)
        previous_indent = indent
    return "\n".join(modified_lines)

In [77]:
# The output of the this step is to a temporary file because it needs to be reviewed before it can be used. Only once the review is
# complete should the file be moved to the ./manual/ folder for later use

document_text = output_doc_as_text(doc, start_page = 6)

# output_file = "./tmp/adla_manual_intermediate.txt"
# with open(output_file, 'w', encoding="utf-8") as f:
#         f.write(document_text)

output_file = "./tmp/adla_manual.txt"
document_text = add_indents(document_text, adla_index)
with open(output_file, 'w', encoding="utf-8") as f:
        f.write(document_text)

Now you need to manually review the file and make the necessary changes.

Load the text file into the data structures to check everything is ok


In [6]:
import sys
sys.path.insert(0, 'E:/Code/pdf/excon/')

import importlib
import src.file_tools
importlib.reload(src.file_tools)
from src.file_tools import read_processed_regs_into_dataframe, get_regulation_detail

import src.tree_tools
importlib.reload(src.tree_tools)
from src.tree_tools import build_tree_for_regulation


dir_path = './manual/'
file_list = []
file_list.append('./manual/adla_manual.txt')
non_text_labels = ['Table', 'Formula', 'Example', 'Definition']


df_adla, non_text = read_processed_regs_into_dataframe(file_list=file_list, valid_index_checker=adla_index, non_text_labels=non_text_labels)
tree_adla = build_tree_for_regulation("ADLA", df_adla, valid_index_checker=adla_index)

In [ ]:
df_adla

Starting at an particular parent node (can be the tree root or any child), this method splits up the branch into sections where the text does not exceed a certain word_count cap.

Initially this is used to set up the base DataFrame using node == root and later it can be used if we want to change the word_limit for a specific piece of regulation

In [21]:
def split_regs_into_chunks(node, df_with_regs, word_limit_per_chunk, index_checker):
    node_list=[]
    node_list = split_tree(node, df_excon, word_limit_per_chunk, index_checker, node_list)

    section_word_count = []
    for node in node_list:
        #subsection_text = get_full_text_for_node(node.full_node_name, df, False)
        subsection_text = get_regulation_detail(node.full_node_name, df_excon, index_checker)
        word_count = len(subsection_text.split(' '))
        section_word_count.append([node.full_node_name, subsection_text, word_count])

    column_names = ['section', 'text', 'word_count']
    return pd.DataFrame(section_word_count, columns=column_names)


In [23]:
node = tree_adla.root
#node = tree_adla.get_node(node_str)
word_limit_per_chunk = 500
df = split_regs_into_chunks(node, df_adla, word_limit_per_chunk, adla_index)


NameError: name 'split_tree' is not defined